### Residual Neural Network

In [5]:
import tensorflow as tf
import numpy as np

##### Hyper parameters

In [6]:
EPOCHS = 10

##### Resisual Unit

In [ ]:
class ResidualUnit(tf.keras.Model):
    def __init__(self, filter_in, filter_out, kernel_size):
        super(ResidualUnit, self).__init__()
        self.bn1 = tf.keras.layer.BatchNormalization()
        self.conv1 = tf.keras.layers.Conv2D(filter_out, kernel_size, padding='same')
        
        self.bn = tf.keras.layer.BatchNormalization()
        self.conv2 = tf.keras.layers.Conv2D(filter_out, kernel_size, padding='same')
        
        if filter_in == filter_out :
            self.identity = lambda x: x
        else :
            self.identity = tf.keras.layers.Conv2D(filter)out, (1,1), padding='same'
    def __call__(self, x, training=False, mask=None):
        h = self.bn1(x) # batch norm은 training 시와 test시 달라져야 한다.
        